## Basic implementation of Google Gemini

In [4]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY)

In [7]:


message = llm.invoke("What is the capital of pakistan")
print(message.content)

The capital of Pakistan is Islamabad.


## Langchain PromptTemplates

In [26]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is the this given language: {content}, Give only one word answer, based on the language only. For Example User Input: test = 100, Output: Javascript")

print(llm.invoke(prompt.format(content = "#include <stdlib.h>")).content)

C


### Langchain Chains

In [27]:
from langchain.chains import LLMChain

llm_chain = LLMChain(llm = llm, prompt = prompt)

print(llm_chain.run("from langchain.prompts import PromptTemplate"))

Python


#### Multiple variables chaining

In [30]:
complex_prompt = PromptTemplate.from_template("What is the capital of the {country} and the currency of the {country}, The city must be {city}")
complex_llm_chain = LLMChain(llm = llm, prompt = complex_prompt)

In [33]:
print(complex_llm_chain.run(country = "India" , city = "Karachi"))

The capital of India is New Delhi, not Karachi.  Karachi is a major city in Pakistan.

The currency of India is the Indian Rupee (INR).


### Sequential Prompt Template

In [46]:
from langchain.chains import SimpleSequentialChain


prompt1 = PromptTemplate.from_template("What is this language: {content}, Give only one word answer, based on the language only. For Example User Input: test = 100, Output: Javascript")

chain1 = LLMChain(llm = llm, prompt = prompt1 , output_key = "language")

prompt2 = PromptTemplate.from_template("Based on the {language} that was found, State few use cases of that language.")
chain2 = LLMChain(llm = llm, prompt = prompt2 , output_key = "use_cases") 

sChain = SimpleSequentialChain(chains=[chain1, chain2] , output_variables=["language", "use_cases"])

print(sChain.run("from langchain.prompts import PromptTemplate"))

Python's versatility leads to a wide range of use cases across various domains. Here are a few prominent examples:

**1. Web Development:**

* **Back-end development:** Python frameworks like Django and Flask simplify the creation of robust and scalable web applications. They handle routing, database interactions, security, and other essential tasks.
* **Web scraping:** Libraries like Beautiful Soup and Scrapy allow for automated extraction of data from websites, useful for market research, price comparison, and content aggregation.

**2. Data Science and Machine Learning:**

* **Data analysis and visualization:** Libraries like Pandas, NumPy, and Matplotlib enable efficient data manipulation, analysis, and creation of insightful visualizations.
* **Machine learning model development:**  Scikit-learn, TensorFlow, and PyTorch provide powerful tools for building and training various machine learning models, from simple linear regression to complex neural networks.
* **Deep learning:** Py